# Data Preparation

In [1]:
!python --version

Python 3.10.12


In [2]:
!pip -qq install synapseclient

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.0/299.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.6 MB/s eta 0:00:00


In [3]:
!wget -qq https://github.com/matomo-org/travis-scripts/raw/master/fonts/Arial.ttf -O arial.ttf
!wget -qq https://github.com/foursquare/foursquair/raw/master/src/assets/fonts/Arial%20Bold.ttf -O arial_bold.ttf

In [4]:
import cv2
import yaml
import math
import random
import pickle
import numpy as np
import pandas as pd
import synapseclient
from enum import Enum
from tqdm import tqdm
from random import seed
from collections import Counter
seed(10)

import networkx as nx
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from PIL import Image, ImageDraw, ImageFont

from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler

from typing import List
from dataclasses import dataclass
from abc import ABC, abstractmethod
from IPython.display import display

pd.set_option('display.max_columns', None)

`config.yaml` should be kept in the same directory of the notebooks and be prepared like this.

    auth_token: YOUR_AUTH_TOKEN_HERE

Please go to synapse and log in to get the authentication token.

In [6]:
def get_auth_token_from_yaml(file_path='config.yaml'):
    with open(file_path, 'r') as file:
        config_data = yaml.safe_load(file)
        auth_token = config_data.get('auth_token')
        return auth_token

auth_token = get_auth_token_from_yaml()
syn = synapseclient.Synapse()
syn.login(authToken=auth_token)

Welcome, fuad021!



INFO:synapseclient_default:Welcome, fuad021!



### Getting the case study data

In [7]:
syn31123117 = syn.get(entity='syn31123117')

raw_df = pd.read_csv(syn31123117.path, sep='\t')
raw_df = raw_df.fillna(0)
raw_df = raw_df.set_index('Gene_symbol')

raw_df

,GLSS.SM.R109.TP.01R.RNA.I4HTPP,GLSS.19.0274.TP.01R.RNA.ZZD8PZ,GLSS.CU.R005.TP.01R.RNA.LOURPF,GLSS.SM.R060.TP.01R.RNA.ZI9YVM,GLSS.MD.0017.R2.01R.RNA.L5HMEO,GLSS.HF.47E9.R1.01R.RNA.7W5GMH,TCGA.TQ.A8XE.TP.11R.RNA.0YPLAZ,GLSS.LU.00C2.TP.01R.RNA.2QRUOC,GLSS.HF.CB66.TP.01R.RNA.9MRJMX,GLSS.LU.00B9.R1.01R.RNA.1IVJNJ,TCGA.06.0210.TP.01R.RNA.JMBENW,TCGA.DU.7304.R1.12R.RNA.OMXRQS,GLSS.SN.0001.R1.01R.RNA.PZAYRK,GLSS.MD.0022.TP.01R.RNA.EDOJNO,GLSS.19.0267.R1.01R.RNA.NCCNA9,TCGA.TQ.A7RK.R2.11R.RNA.BJMJ93,GLSS.HF.DF35.TP.01R.RNA.IH1LVD,TCGA.06.0221.R1.11R.RNA.UCUX3T,GLSS.MD.0048.R1.01R.RNA.RVQU3C,GLSS.HF.6504.TP.01R.RNA.5OSIEV,GLSS.CU.P024.TP.01R.RNA.8JTL48,GLSS.HF.4F0A.TP.01R.RNA.GG6762,GLSS.SM.R107.TP.01R.RNA.ZSYP37,GLSS.CU.R009.R1.01R.RNA.QREUYE,TCGA.FG.5965.R2.11R.RNA.UCBSIY,TCGA.FG.5963.R1.12R.RNA.1EJ1SV,GLSS.CU.P104.R1.01R.RNA.L5UZ2L,GLSS.HF.B30B.R2.01R.RNA.K9X72E,GLSS.MD.0046.R1.01R.RNA.Y7WTHG,GLSS.SN.0017.TP.01R.RNA.FX5HYW,GLSS.MD.0017.R1.01R.RNA.8FU0BP,GLSS.SM.R070.TP.01R.RNA.QWG60O,GLSS.LU.00C7.R1.01R.RNA.898J17,GLSS.SM.R101.R1.01R.RNA.7ATA59,GLSS.MD.0012.R1.01R.RNA.GB101L,GLSS.HF.2869.TP.01R.RNA.2H6MOF,GLSS.MD.0027.R1.01R.RNA.E771C5,GLSS.MD.0044.TP.01R.RNA.2ARML9,GLSS.CU.P069.TP.01R.RNA.CVOV9T,GLSS.SM.R101.TP.02R.RNA.V4TRVD,TCGA.19.0957.R1.01R.RNA.7WHBFU,GLSS.SN.0009.TP.01R.RNA.1DY2G0,GLSS.19.0279.R2.01R.RNA.G733J7,GLSS.HK.0004.R1.01R.RNA.1PAW5U,GLSS.LU.00C7.TP.01R.RNA.7QCLSN,GLSS.19.0271.R1.01R.RNA.ZGQFM3,GLSS.19.0280.R1.01R.RNA.CZ9ZTK,TCGA.DH.A669.TP.12R.RNA.XG7CTS,GLSS.MD.0041.R1.01R.RNA.YF565D,GLSS.CU.R012.TP.01R.RNA.21UZH1,GLSS.CU.P053.TP.01R.RNA.DAMM5M,GLSS.CU.P101.TP.01R.RNA.2II6XG,GLSS.LX.0304.R1.01R.RNA.68YN4U,GLSS.SF.0038.TP.01R.RNA.TSEK1M,GLSS.SM.R083.TP.01R.RNA.RZOKGO,GLSS.SM.R064.TP.01R.RNA.W6PIW2,GLSS.HF.2829.R1.01R.RNA.D71HI8,GLSS.SN.0002.TP.01R.RNA.F7DBXG,GLSS.LU.00C4.R1.01R.RNA.O1YTL6,GLSS.SM.R080.TP.01R.RNA.2N8175,GLSS.MD.0027.R2.01R.RNA.1RO1LP,GLSS.SM.R091.TP.01R.RNA.RUF3X9,GLSS.SM.R106.TP.01R.RNA.P5JI35,GLSS.SN.0010.TP.01R.RNA.E97Z4T,GLSS.CU.R002.TP.01R.RNA.7CO09H,GLSS.MD.LP02.R1.01R.RNA.6ZQS8U,GLSS.SM.R091.R1.01R.RNA.M356AA,TCGA.DU.5870.R1.12R.RNA.GIZUWN,GLSS.SN.0013.TP.01R.RNA.5952RG,GLSS.CU.P102.R1.01R.RNA.QODUFA,GLSS.CU.P013.R3.01R.RNA.V38ZQ5,TCGA.19.4065.TP.01R.RNA.UGJCJV,GLSS.SM.R082.R1.01R.RNA.9RBAL7,GLSS.19.0272.R1.01R.RNA.MW1TDZ,GLSS.MD.0012.TP.01R.RNA.49NNFK,GLSS.MD.0035.R1.01R.RNA.ZXNEZ3,GLSS.SN.0008.R1.01R.RNA.AK537W,GLSS.MD.0049.TP.01R.RNA.HFW0T1,GLSS.MD.0019.TP.01R.RNA.MQF7QS,GLSS.HF.2829.TP.01R.RNA.90I02Y,GLSS.SN.0017.R1.01R.RNA.OEK8D6,TCGA.TQ.A7RK.R1.11R.RNA.PKFXB3,GLSS.HF.9A7A.TP.01R.RNA.KGGESL,GLSS.HF.2919.R1.01R.RNA.R6L8SO,GLSS.LU.0B10.TP.01R.RNA.LNPBXB,TCGA.DU.6397.TP.11R.RNA.NYNT65,GLSS.SM.R093.R1.01R.RNA.56NKHA,TCGA.06.0190.TP.01R.RNA.S2074S,GLSS.HF.F922.R1.01R.RNA.XNFKVB,TCGA.06.0125.TP.01R.RNA.RCQ5QS,GLSS.HF.753F.R4.01R.RNA.B36DGR,GLSS.HF.57AE.R2.01R.RNA.NAVDEK,GLSS.MD.0020.TP.01R.RNA.NVJSCE,GLSS.MD.0009.TP.01R.RNA.TX2UR7,GLSS.HK.0001.TP.01R.RNA.BYEBS0,GLSS.HF.DF35.R1.01R.RNA.KMLJLA,GLSS.SM.R087.TP.01R.RNA.M99T61,GLSS.MD.0026.TP.01R.RNA.7PF2HB,GLSS.HF.3118.R1.01R.RNA.HNBVM0,GLSS.SM.R066.TP.01R.RNA.G9A6CK,TCGA.06.0211.R1.02R.RNA.3YQ1G4,GLSS.SN.0006.TP.01R.RNA.N5ULKZ,GLSS.LX.0083.R1.01R.RNA.VS46JF,GLSS.HF.6504.R1.01R.RNA.BSOQQB,TCGA.06.0152.R1.01R.RNA.1IIR6O,TCGA.DU.6407.R2.11R.RNA.F6PNTL,GLSS.CU.P003.R1.01R.RNA.2OJ4BA,GLSS.SM.R067.TP.01R.RNA.JYXPKW,GLSS.HF.4D9A.TP.01R.RNA.C8NU0J,GLSS.SM.R081.TP.01R.RNA.4GQW1O,GLSS.CU.R001.R1.01R.RNA.SRFKW8,GLSS.LU.00B9.TP.01R.RNA.S9XA8D,GLSS.SM.R061.TP.01R.RNA.AZSPBW,GLSS.MD.0014.R1.01R.RNA.WIUIL4,GLSS.LU.0B13.R1.01R.RNA.CZ1AHH,GLSS.SM.R101.TP.01R.RNA.G7G4Q5,GLSS.HF.9A7A.R1.01R.RNA.RQWI9I,GLSS.SM.R095.TP.01R.RNA.5T2HT4,GLSS.SN.0013.R2.01R.RNA.A9XVF7,GLSS.CU.P021.TP.01R.RNA.Q4LSD8,GLSS.CU.P069.R1.01R.RNA.95RLAP,GLSS.HF.2998.R1.01R.RNA.2HE04E,GLSS.HF.2934.TP.01R.RNA.WUQKHB,GLSS.SM.R107.R1.01R.RNA.ID07M4,GLSS.SM.R107.R1.02R.RNA.1K9XUJ,GLSS.LX.0357.R2.01R.RNA.HBTTUU,GLSS.HF.EE77.R2.01R.RNA.ZNRTM7,GLSS.HK.0003.R1.01R.RNA.34VZDE,GLSS.LX.0083.R2.01R.RNA.EXTF60,

In [8]:
def get_case_df(raw_df, patient_ids, case_name):
    case_ids = sorted([id for id in list(raw_df.columns) if id.split('.')[2] in patient_ids])
    case_df = raw_df[case_ids]
    case_df.columns = [case_name+'_'+column_name.split('.')[3] for column_name in case_df.columns]
    case_df = case_df.groupby(case_df.columns, axis=1).median()
    return case_df

# baseline_ids = ['F922', '9A7A', '57AE', '4F0A', 'CB66']
baseline_ids = ['5965', 'F922', 'A7RK', 'R064', '9A7A']
baseline_df = get_case_df(raw_df, baseline_ids, 'baseline')
aggressive_df = get_case_df(raw_df, ["0279"], 'aggressive')
nonaggressive_df = get_case_df(raw_df, ["0027"], 'nonaggressive')

print(baseline_df.shape, aggressive_df.shape, nonaggressive_df.shape)
baseline_df

(56638, 3) (56638, 3) (56638, 3)


,baseline_R1,baseline_R2,baseline_TP
Gene_symbol,,,
CMSS1,32.506237,41.346870,25.960538
STX16,32.786515,38.846232,22.001576
TLE4,21.343468,26.251004,11.120648
LRRN1,18.381122,18.670567,48.811640
YKT6,95.362065,91.290736,78.563483
...,...,...,...
LAMTOR3P1,0.000000,0.000000,0.000000
RNA5SP345,0.000000,0.000000,0.000000
RN7SKP61,0.000000,0.000000,0.000000


In [9]:
all_case_df = pd.concat([baseline_df, aggressive_df, nonaggressive_df], axis=1)
all_case_df = all_case_df.loc[(all_case_df >= 0.001).all(axis=1)]
all_case_df

,baseline_R1,baseline_R2,baseline_TP,aggressive_R1,aggressive_R2,aggressive_TP,nonaggressive_R1,nonaggressive_R2,nonaggressive_TP
Gene_symbol,,,,,,,,,
CMSS1,32.506237,41.346870,25.960538,26.446038,27.000944,32.025694,31.395261,48.859002,44.117700
STX16,32.786515,38.846232,22.001576,99.860372,104.422161,126.156318,20.050733,21.480300,45.597264
TLE4,21.343468,26.251004,11.120648,23.209358,19.561900,31.615145,72.895632,96.452925,65.074070
LRRN1,18.381122,18.670567,48.811640,2.167500,0.433401,5.336610,121.089577,101.425790,137.481300
YKT6,95.362065,91.290736,78.563483,52.902599,65.192139,59.786585,11.058250,13.540128,9.911100
...,...,...,...,...,...,...,...,...,...
UQCRFS1,75.973600,64.389700,65.626800,54.498700,58.399200,50.797400,68.277000,105.298000,60.563400
C4orf32,4.120360,5.598950,3.359910,0.268489,0.327187,0.151905,4.348520,4.666130,4.126970
MTRNR2L9,4.082070,7.589460,25.170100,0.430776,0.616574,0.785486,111.565000,63.872200,98.041800


In [10]:
baseline_df = all_case_df[all_case_df.columns[all_case_df.columns.str.contains("baseline")]]
aggressive_df = all_case_df[all_case_df.columns[all_case_df.columns.str.contains("aggressive")]]
nonaggressive_df = all_case_df[all_case_df.columns[all_case_df.columns.str.contains("nonaggressive")]]

print(baseline_df.shape, aggressive_df.shape, nonaggressive_df.shape)

(11075, 3) (11075, 6) (11075, 3)


### Differentially Expressed Gene Selection

In [11]:
def calculate_FC(df):
    name = df.columns[0].split('_')[0]
    temp_df = df.copy()
    temp_df.loc[:, f"{name}_R1/TP"] = temp_df[f"{name}_R1"] / temp_df[f"{name}_TP"]
    temp_df.loc[:, f"{name}_R2/TP"] = temp_df[f"{name}_R2"] / temp_df[f"{name}_TP"]
    return temp_df

baseline_df = calculate_FC(baseline_df)
aggressive_df = calculate_FC(aggressive_df)
nonaggressive_df = calculate_FC(nonaggressive_df)

print(baseline_df.shape, aggressive_df.shape, nonaggressive_df.shape)
nonaggressive_df

(11075, 5) (11075, 8) (11075, 5)


,nonaggressive_R1,nonaggressive_R2,nonaggressive_TP,nonaggressive_R1/TP,nonaggressive_R2/TP
Gene_symbol,,,,,
CMSS1,31.395261,48.859002,44.117700,0.711625,1.107469
STX16,20.050733,21.480300,45.597264,0.439735,0.471087
TLE4,72.895632,96.452925,65.074070,1.120195,1.482202
LRRN1,121.089577,101.425790,137.481300,0.880771,0.737742
YKT6,11.058250,13.540128,9.911100,1.115744,1.366158
...,...,...,...,...,...
UQCRFS1,68.277000,105.298000,60.563400,1.127364,1.738641
C4orf32,4.348520,4.666130,4.126970,1.053683,1.130643
MTRNR2L9,111.565000,63.872200,98.041800,1.137933,0.651479


In [12]:
def get_degs(df, c, up_th, down_th):
    glist_up = list(df[df[c] >= up_th].index)
    glist_down = list(df[df[c] <= dn_th].index)
    glist = set(glist_up + glist_down)
    print(f"# selected genes: up:{len(glist_up)}, down:{len(glist_down)}, total:{len(glist)}")
    return glist

# selecting differentiall expressed genes if FC >= 1.5 or FC =< 0.5
up_th = 1.5
dn_th = abs(1-(up_th-1))
print(f"up-regulation th: {up_th}\ndonw-regulation th: {dn_th}\n")

gene_sets = []
for df in [baseline_df, aggressive_df, nonaggressive_df]:
    name = df.columns[0].split('_')[0]
    for c in [f"{name}_R1/TP", f"{name}_R2/TP"]:
        gene_sets.append(get_degs(df, c, up_th, dn_th))

with open('genesets_FC.pkl', 'wb') as f:
    pickle.dump(gene_sets, f)

up-regulation th: 1.5
donw-regulation th: 0.5

# selected genes: up:2100, down:438, total:2538
# selected genes: up:3461, down:317, total:3778
# selected genes: up:741, down:922, total:1663
# selected genes: up:980, down:1100, total:2080
# selected genes: up:1727, down:3196, total:4923
# selected genes: up:2840, down:2570, total:5410


In [15]:
with open('genesets_FC.pkl', 'rb') as f:
    gene_sets = pickle.load(f)

set_baseline_R1TP = set(gene_sets[0])
set_baseline_R2TP = set(gene_sets[1])
set_aggressive_R1TP = set(gene_sets[2])
set_aggressive_R2TP = set(gene_sets[3])
set_nonaggressive_R1TP = set(gene_sets[4])
set_nonaggressive_R2TP = set(gene_sets[5])

len(set_baseline_R1TP), len(set_baseline_R2TP), len(set_aggressive_R1TP), len(set_aggressive_R2TP), len(set_nonaggressive_R1TP), len(set_nonaggressive_R2TP)

(2538, 3778, 1663, 2080, 4923, 5410)

### Retrieving PAGs for DEGs using PAGER API

In [16]:
!wget -qq http://discovery.informatics.uab.edu/PAGER/application/download/PAGER.py

In [ ]:
import PAGER as pager
PAGER = pager.PAGER()

def get_PAGs(gene_list):
    PAG_result = PAGER.run_pager(
        gene_list,
        source = ["WikiPathway_2021"],
        Type='P',
        minSize=1,
        maxSize=2000,
        similarity = 0.05,
        overlap = 1,
        nCoCo = 0,
        pValue = 0.05,
        FDR = 0.5
    )
    return PAG_result

pag_bR1 = get_PAGs(set_baseline_R1TP)
pag_bR2 = get_PAGs(set_baseline_R2TP)
pag_aR1 = get_PAGs(set_aggressive_R1TP)
pag_aR2 = get_PAGs(set_aggressive_R2TP)
pag_nR1 = get_PAGs(set_nonaggressive_R1TP)
pag_nR2 = get_PAGs(set_nonaggressive_R2TP)

print(pag_bR1.shape, pag_bR2.shape, pag_aR1.shape, pag_aR2.shape, pag_nR1.shape, pag_nR2.shape)
pag_bR1.head()

(247, 15) (432, 15) (118, 15) (140, 15) (459, 15) (558, 15)


,GS_ID,NAME,SOURCE,GS_SIZE,ORGANISM,DESCRIPTION,LINK,TYPE,MULTI_N,OLAP,COCO_V2,SIMILARITY_SCORE,pvalue,Rank,pFDR
0,WAG002718,Neutrophil degranulation,WikiPathway_2021,474,Homo sapiens,Neutrophil degranulation,https://www.wikipathways.org/index.php/Pathway...,P,1333,120,172.7292014,.0838279321392930300961184936183623089448,1.732402e-44,1,5.422417e-42
1,WAG002628,VEGFA-VEGFR2 signaling pathway,WikiPathway_2021,438,Homo sapiens,VEGFA-VEGFR2 signaling pathway,https://www.wikipathways.org/index.php/Pathway...,P,1333,97,980.8100106,.0712460408468820826612487529758081271878,3.931521e-31,2,1.226635e-28
2,WAG003066,Endothelin pathway,WikiPathway_2021,194,Homo sapiens,Endothelin pathway,https://www.wikipathways.org/index.php/Pathway...,P,1333,56,968.3903924,.0744733638450988550382938067279983973618,1.531451e-24,3,4.762811e-22
3,WAG002670,TYROBP causal network in microglia,WikiPathway_2021,60,Homo sapiens,TYROBP causal network in microglia,https://www.wikipathways.org/index.php/Pathway...,P,1333,32,221.1523998,.1166432839698623018965965185762535723565,3.132260e-24,4,9.710006e-22
4,WAG002820,Ebola virus pathway in host,WikiPathway_2021,131,Homo sapiens,Ebola virus pathway in host,https://www.wikipathways.org/index.php/Pathway...,P,1333,45,650.3115529,.082421802271457829082107614969279463787,1.841119e-23,5,5.689056e-21


In [ ]:
pag_bR1.to_csv("pag_bR1_before_DPA.csv")
pag_bR2.to_csv("pag_bR2_before_DPA.csv")
pag_aR1.to_csv("pag_aR1_before_DPA.csv")
pag_aR2.to_csv("pag_aR2_before_DPA.csv")
pag_nR1.to_csv("pag_nR1_before_DPA.csv")
pag_nR2.to_csv("pag_nR2_before_DPA.csv")

!zip -r pag_before_DPA.zip *_before_DPA.csv

  adding: pag_aR1_before_DPA.csv (deflated 71%)
  adding: pag_aR2_before_DPA.csv (deflated 70%)
  adding: pag_bR1_before_DPA.csv (deflated 71%)
  adding: pag_bR2_before_DPA.csv (deflated 71%)
  adding: pag_nR1_before_DPA.csv (deflated 71%)
  adding: pag_nR2_before_DPA.csv (deflated 71%)


### m-type PAG-2-PAG relationships

In [ ]:
def get_m_relationships(pag_df):
    """retrieve the m-type relationships of PAGs using a list of PAG IDs"""
    return PAGER.pathInt(list(pag_df["GS_ID"]))

m_bR1 = get_m_relationships(pag_bR1)
m_bR2 = get_m_relationships(pag_bR2)
m_aR1 = get_m_relationships(pag_aR1)
m_aR2 = get_m_relationships(pag_aR2)
m_nR1 = get_m_relationships(pag_nR1)
m_nR2 = get_m_relationships(pag_nR2)

print(m_bR1.shape, m_bR2.shape, m_aR1.shape, m_aR2.shape, m_nR1.shape, m_nR2.shape)
m_bR1.head()

(948, 7) (3978, 7) (210, 7) (256, 7) (5500, 7) (7590, 7)


,GS_A_ID,GS_A_SIZE,GS_B_ID,GS_B_SIZE,OLAP,SIMILARITY,PVALUE
0,WAG002588,38,WAG002440,21,8,.2016806722689076,43.262700411608016
1,WAG002805,341,WAG002161,83,43,.19390317174208646,147.61739136463984
2,WAG002596,246,WAG002683,29,16,.15976567700705632,62.03314298178884
3,WAG002596,246,WAG002118,55,31,.20457912457912455,120.66116349772959
4,WAG002345,97,WAG003215,107,35,.2378454218263893,148.4504182388598


In [ ]:
for m in [m_bR1, m_bR2, m_aR1, m_aR2, m_nR1, m_nR2]:
    print(f'Unique PAGs: {len(set(m["GS_A_ID"]))}')

m_bR1.to_csv("m_bR1.csv", index=False)
m_bR2.to_csv("m_bR2.csv", index=False)
m_aR1.to_csv("m_aR1.csv", index=False)
m_aR2.to_csv("m_aR2.csv", index=False)
m_nR1.to_csv("m_nR1.csv", index=False)
m_nR2.to_csv("m_nR2.csv", index=False)

!zip -r membership.zip m_*.csv

Unique PAGs: 189
Unique PAGs: 366
Unique PAGs: 67
Unique PAGs: 91
Unique PAGs: 407
Unique PAGs: 490
  adding: m_aR1.csv (deflated 72%)
  adding: m_aR2.csv (deflated 71%)
  adding: m_bR1.csv (deflated 69%)
  adding: m_bR2.csv (deflated 65%)
  adding: m_nR1.csv (deflated 64%)
  adding: m_nR2.csv (deflated 63%)


### Differential PAG analysis


$n = NumberOfGenesInPAG$ \\
$W_i = RPScore(Gene_i)$ \\
$FC_i = FoldChange(Gene_i)$ \\

Calculating weighted FC for each PAG with the equation below: \\

$PAG_{WFC} = \frac{\sum_{i=1}^n W_i * FC_i}{\sum_{i=1}^n W_i}$ \\


In [ ]:
%%time

def weightedFC(wfc, w):
    if len(wfc):
        return round(sum(wfc)/sum(w), 4)
    else:
        return float(0)

color_scheme = {
    "up": "red",
    "down": "blue",
    "between": "yellow",
    "insignificant": "black",
}

def get_pagFC(pag_df, mem_df, fc_df, fc_col, up_th, dn_th):
    pag_IDs = []
    pag_wfc = []
    pag_pvalue = []
    pag_pFDR = []

    # retrieve the membership of PAGs using a list of PAG IDs
    PAG_member = PAGER.pathMember(list(set(pag_df["GS_ID"])))

    for pagID in tqdm(PAG_member["GS_ID"].unique()):
        # retrieve RP-ranked genes with RP-score of the given PAG_IDs
        PAGRankedGenes = PAGER.pagRankedGene(pagID)
        PAGRankedGenes["RP_SCORE"] = PAGRankedGenes["RP_SCORE"].astype(float)
        # print(f'avg RP score: {np.mean(PAGRankedGenes["RP_SCORE"])}')

        # calculating pagFCs
        ws = []
        wfc = []
        for gene in PAGRankedGenes["GENE_SYM"]:
            if gene in fc_df.index:
                w = PAGRankedGenes[PAGRankedGenes["GENE_SYM"]==gene]["RP_SCORE"].values[0]
                fc = fc_df[fc_df.index==gene][fc_col].values[0]
                if not np.isnan(w):
                    ws.append(w)
                    wfc.append(w*fc)

        pag_IDs.append(pagID)
        pag_wfc.append(weightedFC(wfc, ws))
        pag_pvalue.append(pag_df[pag_df["GS_ID"]==pagID]["pvalue"].values[0])
        pag_pFDR.append(pag_df[pag_df["GS_ID"]==pagID]["pFDR"].values[0])

    pagFC_df = pd.DataFrame({
        "GS_ID": pag_IDs,
        "wfc": pag_wfc,
        "pvalue": pag_pvalue,
        "pFDR": pag_pFDR
    })

    pag_colors = []
    print(f'# ! pFDR [BLACK]: {pagFC_df[pagFC_df["pFDR"] >= 0.05].shape}')

    for i, row in pagFC_df.iterrows():
        if row["pFDR"] < 0.05:
            if row["wfc"] >= up_th:
                pag_colors.append("red")
            elif row["wfc"] <= dn_th:
                pag_colors.append("blue")
            else:
                pag_colors.append("yellow")
        else:
            pag_colors.append("black")
    pagFC_df["color"] = pag_colors

    print("\nBEFORE FILTERING")
    print(f'shape: {pagFC_df.shape}\tmin: {min(pagFC_df["wfc"])}\tmax: {max(pagFC_df["wfc"])}')
    print(f'{pagFC_df["color"].value_counts()}\n')

    sel_dfs = []
    assert ['red', 'blue', 'yellow', 'black'] == list(color_scheme.values())
    for col in ['red', 'blue', 'yellow', 'black']:
        sel_dfs.append(pagFC_df[pagFC_df["color"]==col].sort_values(by='pvalue'))       # .sort_values(by='pvalue').head(15)
    pagFC_df = pd.concat(sel_dfs, ignore_index=True).sort_values(by='pvalue')

    print("AFTER FILTERING")
    print(f'shape: {pagFC_df.shape}\tmin: {min(pagFC_df["wfc"])}\tmax: {max(pagFC_df["wfc"])}')
    print(f'{pagFC_df["color"].value_counts()}\n')

    pagFC_df["log_wfc"] = np.log2(pagFC_df["wfc"])
    pagFC_df["abs_log_wfc"] = abs(np.log2(pagFC_df["wfc"]))
    pagFC_df.reset_index(inplace=True, drop=True)
    pagFC_df.to_csv(f"pag_{fc_col.replace('/', '')[:-2]}.csv", index=False)
    return pagFC_df

up_th = 1.25
dn_th = abs(1-(up_th-1))
print(f"up-regulation th: {up_th}\ndonw-regulation th: {dn_th}\n")

pag_bR1 = get_pagFC(pag_bR1, m_bR1, baseline_df, "baseline_R1/TP", up_th, dn_th)
pag_bR2 = get_pagFC(pag_bR2, m_bR2, baseline_df, "baseline_R2/TP", up_th, dn_th)
pag_aR1 = get_pagFC(pag_aR1, m_aR1, aggressive_df, "aggressive_R1/TP", up_th, dn_th)
pag_aR2 = get_pagFC(pag_aR2, m_aR2, aggressive_df, "aggressive_R2/TP", up_th, dn_th)
pag_nR1 = get_pagFC(pag_nR1, m_nR1, nonaggressive_df, "nonaggressive_R1/TP", up_th, dn_th)
pag_nR2 = get_pagFC(pag_nR2, m_nR2, nonaggressive_df, "nonaggressive_R2/TP", up_th, dn_th)

up-regulation th: 1.25
donw-regulation th: 0.75



100%|██████████| 247/247 [02:37<00:00,  1.57it/s]


# ! pFDR [BLACK]: (44, 4)

BEFORE FILTERING
shape: (247, 5)	min: 0.7056	max: 5.023
red       182
black      44
yellow     21
Name: color, dtype: int64

AFTER FILTERING
shape: (247, 5)	min: 0.7056	max: 5.023
red       182
black      44
yellow     21
Name: color, dtype: int64



100%|██████████| 432/432 [04:29<00:00,  1.60it/s]


# ! pFDR [BLACK]: (106, 4)

BEFORE FILTERING
shape: (432, 5)	min: 0.656	max: 9.8722
red       320
black     106
yellow      6
Name: color, dtype: int64

AFTER FILTERING
shape: (432, 5)	min: 0.656	max: 9.8722
red       320
black     106
yellow      6
Name: color, dtype: int64



...

# ! pFDR [BLACK]: (26, 4)

BEFORE FILTERING
shape: (140, 5)	min: 0.0	max: 34.0267
yellow    63
red       34
black     26
blue      17
Name: color, dtype: int64

AFTER FILTERING
shape: (140, 5)	min: 0.0	max: 34.0267
yellow    63
red       34
black     26
blue      17
Name: color, dtype: int64



100%|██████████| 459/459 [04:56<00:00,  1.55it/s]
[WARNING] /usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning:

divide by zero encountered in log2



divide by zero encountered in log2


[WARNING] /usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning:

divide by zero encountered in log2



divide by zero encountered in log2




# ! pFDR [BLACK]: (126, 4)

BEFORE FILTERING
shape: (459, 5)	min: 0.0	max: 3.4701
yellow    278
black     126
blue       44
red        11
Name: color, dtype: int64

AFTER FILTERING
shape: (459, 5)	min: 0.0	max: 3.4701
yellow    278
black     126
blue       44
red        11
Name: color, dtype: int64



100%|██████████| 558/558 [05:53<00:00,  1.58it/s]

# ! pFDR [BLACK]: (133, 4)

BEFORE FILTERING
shape: (558, 5)	min: 0.1613	max: 3.0221
red       230
yellow    178
black     133
blue       17
Name: color, dtype: int64

AFTER FILTERING
shape: (558, 5)	min: 0.1613	max: 3.0221
red       230
yellow    178
black     133
blue       17
Name: color, dtype: int64

CPU times: user 3min 20s, sys: 4.01 s, total: 3min 24s
Wall time: 20min 31s


In [ ]:
pag_bR1.to_csv("pag_bR1_after_DPA.csv")
pag_bR2.to_csv("pag_bR2_after_DPA.csv")
pag_aR1.to_csv("pag_aR1_after_DPA.csv")
pag_aR2.to_csv("pag_aR2_after_DPA.csv")
pag_nR1.to_csv("pag_nR1_after_DPA.csv")
pag_nR2.to_csv("pag_nR2_after_DPA.csv")

!zip -r pag_after_DPA.zip *_after_DPA.csv

  adding: pag_aR1_after_DPA.csv (deflated 62%)
  adding: pag_aR2_after_DPA.csv (deflated 62%)
  adding: pag_bR1_after_DPA.csv (deflated 62%)
  adding: pag_bR2_after_DPA.csv (deflated 62%)
  adding: pag_nR1_after_DPA.csv (deflated 63%)
  adding: pag_nR2_after_DPA.csv (deflated 63%)
